In [1]:
from collections import defaultdict
import os
import re
import jieba
import codecs

In [7]:
"""import csv
import pandas as pd
stop_word = pd.read_csv(r"C:\Users\CANDY\代码\停用词.txt",quoting=csv.QUOTE_NONE,sep='\t',encoding='utf-8',names=['w'])
stoplist = list(stop_word.w)
stoplist"""

In [13]:
"""#生成stopword表，需要去除一些否定词和程度词汇
stopwords = set()
fr = open(r'C:\Users\CANDY\代码\停用词.txt','r',encoding='utf-8')
for word in fr:
    stopwords.add(word.strip())#Python strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
#读取否定词文件
not_word_file = open(r'C:\Users\CANDY\代码\否定词.txt','r+',encoding='utf-8')
not_word_list = not_word_file.readlines()
not_word_list = [w.strip() for w in not_word_list]
#读取程度副词文件
degree_file = open(r'C:\Users\CANDY\代码\程度副词.txt','r+',encoding='utf-8')
degree_list = degree_file.readlines()
degree_list = [item.split(',')[0] for item in degree_list]
#生成新的停用词表
with open('stopwords.txt','w',encoding='utf-8') as f:
    for word in stopwords:
        if(word not in not_word_list) and (word not in degree_list):
            f.write(word+'\n')"""

In [2]:
#jieba分词后去除停用词
def seg_word(sentence):
    seg_list = jieba.cut(sentence)
    seg_result = []
    for i in seg_list:
        seg_result.append(i)
    stopwords = set()
    with open('stopwords.txt','r',encoding='utf-8') as fr:
        for i in fr:
            stopwords.add(i.strip())
    return list(filter(lambda x :x not in stopwords,seg_result))

In [3]:
#找出文本中的情感词、否定词和程度副词
def classify_words(word_list):
    #读取情感词典文件
    sen_file = open(r'C:\Users\CANDY\代码\BosonNLP_sentiment_score.txt','r+',encoding='utf-8')
    #获取词典文件内容
    sen_list = sen_file.readlines()
    #创建情感字典
    sen_dict = defaultdict()
    #读取词典每一行的内容，将其转换成字典对象，key为情感词，value为其对应的权重
    for i in sen_list:
        if len(i.split(' '))==2:
            sen_dict[i.split(' ')[0]] = i.split(' ')[1]
 
    #读取否定词文件
    not_word_file = open(r'C:\Users\CANDY\代码\否定词.txt','r+',encoding='utf-8')
    not_word_list = not_word_file.readlines()
    #读取程度副词文件
    degree_file = open(r'C:\Users\CANDY\代码\程度副词.txt','r+',encoding='utf-8')
    degree_list = degree_file.readlines()
    degree_dict = defaultdict()
    for i in degree_list:
        degree_dict[i.split(',')[0]] = i.split(',')[1]
 
	sen_word = dict()
	not_word = dict()
	degree_word = dict()
	#分类
	for i in range(len(word_list)):
		word = word_list[i]
		if word in sen_dict.keys() and word not in not_word_list and word not in degree_dict.keys():
			# 找出分词结果中在情感字典中的词
			sen_word[i] = sen_dict[word]
		elif word in not_word_list and word not in degree_dict.keys():
			# 分词结果中在否定词列表中的词
			not_word[i] = -1
		elif word in degree_dict.keys():
			# 分词结果中在程度副词中的词
			degree_word[i]  = degree_dict[word]
 
 
	#关闭打开的文件
	sen_file.close()
	not_word_file.close()
	degree_file.close()
	#返回分类结果
	return sen_word,not_word,degree_word

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-3-44a7a9a8f941>, line 24)

In [22]:
#计算情感词的分数
def score_sentiment(sen_word,not_word,degree_word,seg_result):
    #权重初始化为1
    W = 1
    score = 0
    #情感词下标初始化
    sentiment_index = -1
    #情感词的位置下标集合
    sentiment_index_list = list(sen_word.keys())
    #遍历分词结果
    for i in range(0,len(seg_result)):
        #如果是情感词
        if i in sen_word.keys():
            #权重*情感词得分
            score += W*float(sen_word[i])
            #情感词下标加一，获取下一个情感词的位置
            sentiment_index += 1
            if sentiment_index < len(sentiment_index_list)-1:
                #判断当前的情感词与下一个情感词之间是否有程度副词或否定词
                for j in range(sentiment_index_list[sentiment_index],sentiment_index_list[sentiment_index+1]):
                    #更新权重，如果有否定词，权重取反
                    if j in not_word.keys():
                        W *= -1
                    elif j in degree_word.keys():
                        W *= float(degree_word[j])
        #定位到下一个情感词
        if sentiment_index < len(sentiment_index_list)-1:
            i = sentiment_index_list[sentiment_index+1]
    return score
 
#计算得分
def sentiment_score(sentence):
    #1.对文档分词
    seg_list = seg_word(sentence)
    #2.将分词结果转换成字典，找出情感词、否定词和程度副词
    sen_word,not_word,degree_word = classify_words(seg_list)
    #3.计算得分
    score = score_sentiment(sen_word,not_word,degree_word,seg_list)
    return score
 
print("我今天很高兴也非常开心    ",sentiment_score("我今天很高兴也非常开心"))
print('天灰蒙蒙的，路上有只流浪狗，旁边是破旧不堪的老房子   ',sentiment_score('天灰蒙蒙的，路上有只流浪狗，旁边是破旧不堪的老房子'))
print('愤怒、悲伤和埋怨解决不了问题    ',sentiment_score('愤怒、悲伤和埋怨解决不了问题'))
print('要每天都开心快乐    ',sentiment_score('要每天都开心快乐'))
print('我不喜欢这个世界，我只喜欢你    ',sentiment_score('我不喜欢这个世界，我只喜欢你'))

我今天很高兴也非常开心     2.61234173173
天灰蒙蒙的，路上有只流浪狗，旁边是破旧不堪的老房子    -1.8760024465746
愤怒、悲伤和埋怨解决不了问题     -3.369079988548
要每天都开心快乐     6.79232134331
我不喜欢这个世界，我只喜欢你     5.367487561038


In [34]:
print('up主真好',sentiment_score('up主真好'))
print('up主nice',sentiment_score('up主nice'))
print('感谢金主爸爸',sentiment_score('感谢金主爸爸'))
print('什么玩意，快跑',sentiment_score('什么玩意，快跑'))
print('假的，骗人',sentiment_score('假的，骗人'))
print('闪子，你变了',sentiment_score('闪子，你变了'))

up主真好 0.859935517196
up主nice 1.82528469071
感谢金主爸爸 2.955561249614
什么玩意，快跑 -0.821220640765
假的，骗人 -1.167456647372
闪子，你变了 -0.0696560125525
